# Import packages

In [1]:
import Gavin.utils.make_network as mn
# from time import time
import pandas as pd
# import oatpy as oat
# from scipy.optimize import linear_sum_assignment
# from sklearn.metrics import jaccard_score
# import csv
# from sklearn.cluster import SpectralClustering
# from mpl_toolkits.mplot3d import Axes3D
# import plotly.express as px
# import pebble
import plotly.graph_objects as go
import networkx as nx
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
import pprint
from collections import Counter
import seaborn as sns
import plotly.graph_objs as go
import pickle
from networkx.algorithms.community.quality import modularity

# Load network and rep cycle list
This process takes about 5 minutes on my computer

In [3]:
fp = "INSERT FILE PATH HERE"  # this should go to the applied_math_grb_64hr_12pr.pickle in the shared google drive
with open(fp, "rb") as file:
    res = pickle.load(file)
pickle_res1 = res
G = pickle_res1["graph"]
big_rep_cycle_list1 = []
for thing in (pickle_res1['optimized']["optimal cycle nodes"].dropna()):
    big_rep_cycle_list1.append(list(thing))

# Find communities

In [4]:
coms = nx.community.louvain_communities(G, seed=12, resolution=2.2)  ## tune these parameters
node_to_com_dict = {}
for i, community in enumerate(coms):  # create a dictionary of {node : community the node is in}
    for node in community:
        node_to_com_dict[node] = i

# Visualizations
### No gradient, unweighted

In [8]:
# re-collapse graph
collapsed_G = nx.Graph()

for i in range(len(coms)):
    collapsed_G.add_node(i)

hole_adjacent_edge_count = {}  # track the number of hole-adjacent edges between communities
total_edge_count = {}

# create a set of cycle edges based on the original node cycles
cycle_edges = set()
for cycle in big_rep_cycle_list1:
    for i in range(len(cycle)):
        u, v = cycle[i], cycle[(i + 1) % len(cycle)]
        community_u = node_to_com_dict[u]
        community_v = node_to_com_dict[v]
        if community_u != community_v:
            cycle_edges.add((community_u, community_v))
            cycle_edges.add((community_v, community_u))

# add the edges between communities
for u, v in G.edges():
    community_u = node_to_com_dict[u]
    community_v = node_to_com_dict[v]
    if community_u != community_v:
        if collapsed_G.has_edge(community_u, community_v):
            collapsed_G[community_u][community_v]['weight'] += 1
        else:
            collapsed_G.add_edge(community_u, community_v, weight=1)
        
        if (community_u, community_v) not in total_edge_count:
            total_edge_count[(community_u, community_v)] = 0
            hole_adjacent_edge_count[(community_u, community_v)] = 0
        
        total_edge_count[(community_u, community_v)] += 1
        
        if (community_u, community_v) in cycle_edges or (community_v, community_u) in cycle_edges:
            hole_adjacent_edge_count[(community_u, community_v)] += 1

# get proportions of hole-adjacency for color gradient
edge_proportions = {edge: hole_adjacent_edge_count[edge] / total_edge_count[edge] for edge in total_edge_count}
#randomly position nodes in 3d space
pos = {i: (np.random.random(), np.random.random(), np.random.random()) for i in collapsed_G.nodes}

# do edge traces
edge_trace = []
for edge in collapsed_G.edges():
    x0, y0, z0 = pos[edge[0]]
    x1, y1, z1 = pos[edge[1]]
    proportion = edge_proportions.get(edge, 0)
    color = f'rgb({int(255 * (1 - proportion))}, 0, {int(255 * proportion)})'
    
    edge_trace.append(go.Scatter3d(
        x=[x0, x1, None],
        y=[y0, y1, None],
        z=[z0, z1, None],
        line=dict(width=5, color=color),
        hoverinfo='none',
        mode='lines'
    ))

non_cycle_edge_count = len([e for e in collapsed_G.edges() if e not in cycle_edges and (e[1], e[0]) not in cycle_edges])
print(f"Number of edges not part of any cycle: {non_cycle_edge_count}")

# do node traces
node_trace = go.Scatter3d(
    x=[pos[node][0] for node in collapsed_G.nodes()],
    y=[pos[node][1] for node in collapsed_G.nodes()],
    z=[pos[node][2] for node in collapsed_G.nodes()],
    mode='markers',
    marker=dict(
        size=7,
        color='black',
    ),
    text=list(collapsed_G.nodes()),
    hoverinfo='text'
)


layout = go.Layout(
    width=800,
    height=800,
    showlegend=False,
    scene=dict(
        xaxis=dict(showbackground=False),
        yaxis=dict(showbackground=False),
        zaxis=dict(showbackground=False)
    ),
)

fig = go.Figure(data=edge_trace + [node_trace], layout=layout)
fig.show()
fig.write_html("nograd.html")

Number of edges not part of any cycle: 5


### Gradient, unweighted

In [9]:
collapsed_G = nx.Graph()

for i in range(len(coms)):
    collapsed_G.add_node(i)

hole_adjacent_edge_count = {}
total_edge_count = {}

original_cycle_edges = set()
for cycle in big_rep_cycle_list1:
    for i in range(len(cycle)):
        u, v = cycle[i], cycle[(i + 1) % len(cycle)]
        original_cycle_edges.add((u, v))
        original_cycle_edges.add((v, u))

for u, v in G.edges():
    community_u = node_to_com_dict[u]
    community_v = node_to_com_dict[v]
    if community_u != community_v:
        if collapsed_G.has_edge(community_u, community_v):
            collapsed_G[community_u][community_v]['weight'] += 1
        else:
            collapsed_G.add_edge(community_u, community_v, weight=1)
        
        if (community_u, community_v) not in total_edge_count:
            total_edge_count[(community_u, community_v)] = 0
            hole_adjacent_edge_count[(community_u, community_v)] = 0
        
        total_edge_count[(community_u, community_v)] += 1
        
        if (u, v) in original_cycle_edges or (v, u) in original_cycle_edges:
            hole_adjacent_edge_count[(community_u, community_v)] += 1

edge_proportions = {edge: hole_adjacent_edge_count[edge] / total_edge_count[edge] for edge in total_edge_count}

pos = {i: (np.random.random(), np.random.random(), np.random.random()) for i in collapsed_G.nodes}

edge_trace = []
for edge in collapsed_G.edges():
    x0, y0, z0 = pos[edge[0]]
    x1, y1, z1 = pos[edge[1]]
    proportion = edge_proportions.get(edge, 0)
    color = f'rgb({int(255 * (1 - proportion))}, 0, {int(255 * proportion)})'
    
    edge_trace.append(go.Scatter3d(
        x=[x0, x1, None],
        y=[y0, y1, None],
        z=[z0, z1, None],
        line=dict(width=5, color=color),
        hoverinfo='none',
        mode='lines'
    ))

# map original cycle edges to community edges
collapsed_cycle_edges = set()
for u, v in original_cycle_edges:
    community_u = node_to_com_dict[u]
    community_v = node_to_com_dict[v]
    if community_u != community_v:
        collapsed_cycle_edges.add((community_u, community_v))
        collapsed_cycle_edges.add((community_v, community_u))

# count edges part of any cycle and not part of any cycle
cycle_edge_count = len([e for e in collapsed_G.edges() if (e in collapsed_cycle_edges or (e[1], e[0]) in collapsed_cycle_edges)])
non_cycle_edge_count = len(collapsed_G.edges()) - cycle_edge_count

# calculate percentages for gradient
total_edges = len(collapsed_G.edges())
percent_non_cycle = (non_cycle_edge_count / total_edges) * 100
percent_cycle = (cycle_edge_count / total_edges) * 100

print(f"Number of edges not part of any cycle: {non_cycle_edge_count} ({percent_non_cycle:.2f}%)")
print(f"Number of edges part of a cycle: {cycle_edge_count} ({percent_cycle:.2f}%)")

node_trace = go.Scatter3d(
    x=[pos[node][0] for node in collapsed_G.nodes()],
    y=[pos[node][1] for node in collapsed_G.nodes()],
    z=[pos[node][2] for node in collapsed_G.nodes()],
    mode='markers',
    marker=dict(
        size=7,
        color='black',
    ),
    text=list(collapsed_G.nodes()),
    hoverinfo='text'
)

# do color bar/key
color_bar_trace = go.Scatter3d(
    x=[None], y=[None], z=[None],
    mode='markers',
    marker=dict(
        size=0,
        color=[0, 1], 
        colorscale=[[0, 'rgb(255, 0, 0)'], [1, 'rgb(0, 0, 255)']],
        colorbar=dict(
            title="Proportion of Hole-Adjacent Edges",
            tickvals=[0, 1],
            ticktext=["0% (Red)", "100% (Blue)"]
        )
    ),
    hoverinfo='none'
)

layout = go.Layout(
    width=1200,
    height=1000,
    showlegend=False,
    scene=dict(
        xaxis=dict(showbackground=False),
        yaxis=dict(showbackground=False),
        zaxis=dict(showbackground=False)
    ),
)

fig = go.Figure(data=edge_trace + [node_trace, color_bar_trace], layout=layout)
fig.show()
fig.write_html("gradient.html")

Number of edges not part of any cycle: 5 (1.02%)
Number of edges part of a cycle: 486 (98.98%)


### Gradient, weighted

In [10]:
collapsed_G = nx.Graph()

for i in range(len(coms)):
    collapsed_G.add_node(i)

hole_adjacent_edge_count = {}
total_edge_count = {}

original_cycle_edges = set()
for cycle in big_rep_cycle_list1:
    for i in range(len(cycle)):
        u, v = cycle[i], cycle[(i + 1) % len(cycle)]
        original_cycle_edges.add((u, v))
        original_cycle_edges.add((v, u))

for u, v in G.edges():
    community_u = node_to_com_dict[u]
    community_v = node_to_com_dict[v]
    if community_u != community_v:
        if collapsed_G.has_edge(community_u, community_v):
            collapsed_G[community_u][community_v]['weight'] += 1
        else:
            collapsed_G.add_edge(community_u, community_v, weight=1)
        
        if (community_u, community_v) not in total_edge_count:
            total_edge_count[(community_u, community_v)] = 0
            hole_adjacent_edge_count[(community_u, community_v)] = 0
        
        total_edge_count[(community_u, community_v)] += 1
        
        if (u, v) in original_cycle_edges or (v, u) in original_cycle_edges:
            hole_adjacent_edge_count[(community_u, community_v)] += 1

edge_proportions = {edge: hole_adjacent_edge_count[edge] / total_edge_count[edge] for edge in total_edge_count}

# get and normalize weight
weights = [data['weight'] for u, v, data in collapsed_G.edges(data=True)]
min_weight = min(weights)
max_weight = max(weights)
norm_weights = {edge: 3 + 12 * (data['weight'] - min_weight) / (max_weight - min_weight) for edge, data in collapsed_G.edges.items()}

pos = {i: (np.random.random(), np.random.random(), np.random.random()) for i in collapsed_G.nodes}

edge_trace = []
for edge in collapsed_G.edges(data=True):
    x0, y0, z0 = pos[edge[0]]
    x1, y1, z1 = pos[edge[1]]
    proportion = edge_proportions.get((edge[0], edge[1]), 0)
    color = f'rgb({int(255 * (1 - proportion))}, 0, {int(255 * proportion)})'
    weight = norm_weights[(edge[0], edge[1])]  # use weight for thickness of edge
    
    edge_trace.append(go.Scatter3d(
        x=[x0, x1, None],
        y=[y0, y1, None],
        z=[z0, z1, None],
        line=dict(width=weight, color=color),  # use weight for thickness
        hoverinfo='none',
        mode='lines'
    ))

collapsed_cycle_edges = set()
for u, v in original_cycle_edges:
    community_u = node_to_com_dict[u]
    community_v = node_to_com_dict[v]
    if community_u != community_v:
        collapsed_cycle_edges.add((community_u, community_v))
        collapsed_cycle_edges.add((community_v, community_u))

cycle_edge_count = len([e for e in collapsed_G.edges() if (e in collapsed_cycle_edges or (e[1], e[0]) in collapsed_cycle_edges)])
non_cycle_edge_count = len(collapsed_G.edges()) - cycle_edge_count

total_edges = len(collapsed_G.edges())
percent_non_cycle = (non_cycle_edge_count / total_edges) * 100
percent_cycle = (cycle_edge_count / total_edges) * 100

print(f"Number of edges not part of any cycle: {non_cycle_edge_count} ({percent_non_cycle:.2f}%)")
print(f"Number of edges part of a cycle: {cycle_edge_count} ({percent_cycle:.2f}%)")

node_trace = go.Scatter3d(
    x=[pos[node][0] for node in collapsed_G.nodes()],
    y=[pos[node][1] for node in collapsed_G.nodes()],
    z=[pos[node][2] for node in collapsed_G.nodes()],
    mode='markers',
    marker=dict(
        size=7,
        color='black',
    ),
    text=list(collapsed_G.nodes()),
    hoverinfo='text'
)

color_bar_trace = go.Scatter3d(
    x=[None], y=[None], z=[None],
    mode='markers',
    marker=dict(
        size=0,
        color=[0, 1],
        colorscale=[[0, 'rgb(255, 0, 0)'], [1, 'rgb(0, 0, 255)']],
        colorbar=dict(
            title="Proportion of Hole-Adjacent Edges",
            tickvals=[0, 1],
            ticktext=["0% (Red)", "100% (Blue)"]
        )
    ),
    hoverinfo='none'
)

layout = go.Layout(
    width=1200,
    height=1000,
    showlegend=False,
    scene=dict(
        xaxis=dict(showbackground=False),
        yaxis=dict(showbackground=False),
        zaxis=dict(showbackground=False)
    ),
)

fig = go.Figure(data=edge_trace + [node_trace, color_bar_trace], layout=layout)
fig.show()
fig.write_html("gradient_weighted.html")

Number of edges not part of any cycle: 5 (1.02%)
Number of edges part of a cycle: 486 (98.98%)
